# Structure answers with Fireworks functions

Fireworks (FW) function calling model allows has the ability to produced structured responses. This is often useful in question answering when you want to not only get the final answer but also supporting evidence, citation, etc.

In this notebook we show how to use an LLM chain which uses FW functions as part of an overall retrieval pipeline.

In [1]:
import os
import requests
import re
import openai

In [2]:
url = "https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt"
content = requests.get(url).content
content = str(content, "utf-8")

In [3]:
# Some clean up
clean_content = content.replace("\n\n", "\n")

In [4]:
client = openai.OpenAI(
    base_url = "https://api.fireworks.ai/inference/v1",
    api_key = os.environ["FW_API_KEY"],
)
model_name = "accounts/fireworks/models/fw-function-call-34b-v0"

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_answer_with_sources",
            "description": "Answer questions from the user while quoting sources.",
            "parameters": {
                "type": "object",
                "properties": {
                  "answer": {
                      "type": "string",
                      "description": "Answer to the question that was asked."
                  },
                  "sources": {
                      "type": "array",
                      "items": {
                          "type": "string",
                          "description": "Source used to answer the question"
                      }
                  }
                },
                "required": ["answer", "sources"],
            }
        }
    }
]
tool_choice = {"type": "function", "function": {"name":"get_answer_with_sources"}}

In [7]:
messages = [
    {"role": "system", "content": f"You are a helpful assistant who is given document with following content: {clean_content}."
     															"Please reply in succinct manner and be truthful in the reply."},
    {"role": "user", "content": "What did the president say about Ketanji Brown Jackson?"}
]


In [8]:
chat_completion = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=tools,
    tool_choice=tool_choice,
    temperature=0.1
)

In [9]:
print(chat_completion.choices[0].message.model_dump_json(indent=4))

{
    "content": null,
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_Z7ha3Pg1yFDYIw0xSNFfsggz",
            "function": {
                "arguments": "{\"answer\": \"President Biden nominated Ketanji Brown Jackson to the Supreme Court on January 31, 2022. She is a Circuit Court of Appeals judge and a consensus builder. She has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.\", \"sources\": [\"https://www.whitehouse.gov/briefing-room/statements-and-releases/2022/01/31/remarks-by-president-biden-announcing-his-nomination-of-ketanji-brown-jackson-to-the-supreme-court/\"]}",
                "name": "get_answer_with_sources"
            },
            "type": "function",
            "index": 0
        }
    ]
}


In [10]:
agent_response = chat_completion.choices[0].message

messages.append(
    {
        "role": agent_response.role,
        "content": "",
        "tool_calls": [
            tool_call.model_dump()
            for tool_call in agent_response.tool_calls
        ]
    }
)

## Using Function Calling in Conversation

Our model currently support multi-turn conversation when using function calling. You can reference previous completions generated by the model to ask more clarifying questions.

In [11]:
messages.append(
    {
        "role": "user",
        "content": "What did he say about her predecessor?"
    }
)
next_chat_completion = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=tools,
    tool_choice=tool_choice,
    temperature=0.1
)

In [12]:
print(next_chat_completion.choices[0].message.model_dump_json(indent=4))

{
    "content": null,
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_YIkwxMhBMXmospPHVT8Ue1wH",
            "function": {
                "arguments": "\"{\\\"answer\\\": \\\"President Biden said that Justice Breyer's legacy of excellence will be carried on by Ketanji Brown Jackson. Justice Breyer is an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. He has served on the Supreme Court for 16 years.\\\", \\\"sources\\\": [\\\"https://www.whitehouse.gov/briefing-room/statements-and-releases/2022/01/31/remarks-by-president-biden-announcing-his-nomination-of-ketanji-brown-jackson-to-the-supreme-court/\\\"]}\"",
                "name": "get_answer_with_sources"
            },
            "type": "function",
            "index": 0
        }
    ]
}


## Modifying the output format to more specific one

During the conversation, some questions might need a more flexible response format. We have flexibility to change that during the conversation.



In [13]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_answer_with_countries",
            "description": "Answer questions from the user while quoting sources.",
            "parameters": {
                "type": "object",
                "properties": {
                  "answer": {
                      "type": "string",
                      "description": "Answer to the question that was asked."
                  },
                  "countries": {
                      "type": "array",
                      "items": {
                          "type": "string",
                      },
                      "description": "countries mentioned in the sources"
                  }
                },
                "required": ["answer", "countries"],
            }
        }
    }
]
tool_choice = {"type": "function", "function": {"name":"get_answer_with_countries"}}

In [14]:
agent_response = next_chat_completion.choices[0].message

messages.append(
    {
        "role": agent_response.role,
        "content": "",
        "tool_calls": [
            tool_call.model_dump()
            for tool_call in agent_response.tool_calls
        ]
    }
)

messages.append(
    {
        "role": "user",
        "content": "What did he say about human traffickers?"
    }
)

In [15]:
chat_completion = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=tools,
    tool_choice=tool_choice,
    temperature=0.1
)

In [16]:
print(chat_completion.choices[0].message.model_dump_json(indent=4))

{
    "content": null,
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_9iUHkOkoSIpef2HcPvaWHK1x",
            "function": {
                "arguments": "\"{\\\"answer\\\": \\\"President Biden said that we can do all this while keeping lit the torch of liberty that has led generations of immigrants to this land. He also said that we can't let the onslaught of state laws targeting transgender Americans and their families be wrong. We must protect access to health care, prevention, treatment, harm reduction, and recovery. We must get rid of outdated rules that stop doctors from prescribing treatments. And we must stop the flow of illicit drugs by working with state and local law enforcement to go after traffickers.\\\", \\\"countries\\\": [\\\"United States\\\"]}\"",
                "name": "get_answer_with_countries"
            },
            "type": "function",
            "index": 0
        }
    ]
}
